In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [12]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Pernambuco - Produção de Cimento (t)', axis=1)
data

,Ano,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Consumo de Cimento (t)
0,2008,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,...,8.049769,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063,1717.153000
1,2009,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,...,8.078293,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534,1878.938000
2,2010,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,...,8.042187,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189,2406.931000
3,2011,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,...,8.028882,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199,2496.113000
4,2012,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,...,12.192814,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413,2728.230000
5,2013,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,...,13.117303,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013,2982.021000
6,2014,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,...,13.857509,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,2983.632857
7,2015,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,...,14.543115,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,2632.837886
8,2016,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,...,14.833751,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,2308.455257
9,2017,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,...,14.580259,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,1813.941000


In [13]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,8.049769,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063
1,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,8.078293,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534
2,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,8.042187,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189
3,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,8.028882,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199
4,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,12.192814,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413
5,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,13.117303,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013
6,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,13.857509,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118
7,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,14.543115,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579
8,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,14.833751,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812
9,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,14.580259,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
12    2022.411000
13    1940.927000
14            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.894045,1.016308,-1.508819,-2.131804,-1.583249,-1.530584,-1.598164,1.204752,-0.990234,0.166091,...,0.984445,-1.373812,1.355095,1.532738,-1.179403,-0.231175,0.290203,-0.299372,-1.157338,1.628734
1,-1.694920,1.229887,-0.890758,-1.702987,-1.220442,-1.140159,-1.290136,0.820657,-0.546688,0.428911,...,-0.051710,-1.363879,1.312892,1.177660,-1.613673,-0.847777,0.336507,0.759984,0.359687,0.803272
2,-1.063798,1.314665,0.449724,-0.423797,-0.905536,-0.821949,-0.994665,0.275392,0.158672,0.847995,...,-0.115145,-1.376452,1.172184,1.020114,-1.540643,-1.245561,-0.868108,1.779145,1.889008,0.320744
3,-0.515495,1.139248,1.171687,0.366041,-0.637043,-0.574308,-0.711867,-0.132596,0.824346,1.230291,...,0.633403,-1.381086,0.934380,0.438242,-0.718070,-1.254018,-1.004513,1.614369,1.918773,0.127477
4,-0.028247,0.749420,1.471589,0.937804,-0.491590,-0.416427,-0.486099,-0.104860,1.148829,1.382743,...,-0.646476,0.068995,0.412716,0.631847,-0.195530,-1.252337,-1.248108,-0.294581,0.276978,-0.302714
5,0.353986,0.225440,1.211705,1.102177,-0.176084,-0.281472,-0.141927,0.657046,0.830615,1.112297,...,-0.757614,0.390946,0.301975,0.675218,0.448231,-0.576002,-1.498756,-1.259768,-0.603559,-0.884651
6,0.637781,-0.389615,0.795877,0.989019,-0.034819,-0.228569,0.090888,1.615453,-0.215960,0.282052,...,0.345772,0.648721,-0.176426,-0.090537,1.193372,0.489088,-0.569662,-1.695614,-1.361137,-1.146844
7,0.807393,-0.839081,0.097662,0.625383,-0.042209,-0.252893,0.259082,0.558230,-0.940033,-0.632425,...,1.298413,0.887482,-1.082994,-0.722687,1.275456,1.105111,0.690753,-0.893603,-0.968421,-0.951240
8,0.945883,-1.118984,-0.131209,0.524538,1.322915,1.472501,1.131361,-0.835563,-1.089341,-1.093812,...,1.596124,0.988695,-0.668215,-1.368038,0.961643,1.452098,0.570944,-0.088137,-0.150117,-0.905473
9,0.973351,-1.179977,-0.300688,0.396558,1.307025,1.422494,1.201669,-1.495939,-1.153056,-1.335633,...,-0.048438,0.900417,-1.017629,-1.395719,0.796001,1.401274,0.152257,-0.147914,-0.056209,-1.044017


In [16]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,-2.260447,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739


In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [19]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1222385943, 1172177270, 1539606971, 1040885590, 856407370, 2678598909, 1750327404, 2870390555, 4164756588, 1857286843, 3847589102, 4117896417, 1476739211, 3334413531, 2057618005, 3812789736, 139361138, 2547508991, 3648389944, 552231938, 2185817838, 1469127733, 1709170869, 3297258097, 1348407972, 1725437472, 448195723, 929412223, 4241618077, 1722936490, 1885042593, 3268397022, 3976682489, 1123140867, 72575411, 2272339968, 2224631813, 57374849, 4036280448, 606785063, 860384671, 3860630952, 1278306080, 1496928086, 1530723236, 2007443053, 2618300841, 1073844213, 1821692525, 318297012, 3129645196, 4263449007, 3646215439, 508231677, 557958129, 281787030, 2008509070, 2296569847, 1924917652, 444305471, 1204828775, 3286824663, 3354152994, 936712826, 619609225, 4236256287, 394798651, 453437920, 2321432364, 670096282, 1776160831, 1328757832, 2062526565, 3057627671, 3522340089, 1491827293, 501934742, 2687980903, 460810378, 596904023, 2569937489, 117698382, 1162185456, 916033875, 229352158, 220571

2023-09-15 16:18:24.336671: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 750.8348999023438


Step: 19 ___________________________________________
val_loss: 3070.36669921875


Step: 20 ___________________________________________
val_loss: 4.040629863739014


Step: 21 ___________________________________________
val_loss: 0.00028790533542633057


Step: 22 ___________________________________________
val_loss: 172.743408203125


Step: 23 ___________________________________________
val_loss: 3002.4833984375


Step: 24 ___________________________________________
val_loss: 9.5367431640625e-07
winner_seed: 1348407972


Step: 25 ___________________________________________
val_loss: 8015.01416015625


Step: 26 ___________________________________________
val_loss: 1.4162510633468628


Step: 27 ___________________________________________
val_loss: 8702.166015625


Step: 28 ___________________________________________
val_loss: 0.00022912025451660156


Step: 29 ___________________________________________
val_loss: 0.0829935073852539


Step: 30 __________________

In [23]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 473ms/step - loss: 5499324.5000 - val_loss: 3859768.2500
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 5476604.5000 - val_loss: 4151869.7500
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 4835466.5000 - val_loss: 4704302.5000
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 4811065.5000 - val_loss: 4578544.5000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 5036102.0000 - val_loss: 4543909.5000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 4963481.5000 - val_loss: 4840337.5000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 4688639.5000 - val_loss: 5314637.0000
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 4483673.5000 - val_loss: 4229854.0000
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 4631019.5000 - val_loss: 4554

1/1 [==============================] - 0s 15ms/step - loss: 1409842.3750 - val_loss: 1284946.0000
Epoch 74/10000
1/1 [==============================] - 0s 15ms/step - loss: 1332870.6250 - val_loss: 1213018.0000
Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 1340503.3750 - val_loss: 1232593.2500
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 1312048.6250 - val_loss: 1138523.6250
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 1288057.5000 - val_loss: 1134952.8750
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 1242849.6250 - val_loss: 1168247.5000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1243430.8750 - val_loss: 1069909.3750
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1191318.5000 - val_loss: 1120162.7500
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 1200031.5000 - val_loss: 1028728.812

1/1 [==============================] - 0s 15ms/step - loss: 423365.5000 - val_loss: 282350.6875
Epoch 147/10000
1/1 [==============================] - 0s 19ms/step - loss: 417869.9062 - val_loss: 276777.0625
Epoch 148/10000
1/1 [==============================] - 0s 15ms/step - loss: 399777.7812 - val_loss: 291886.9375
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 407013.4062 - val_loss: 260904.2188
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 401284.7812 - val_loss: 262582.1875
Epoch 151/10000
1/1 [==============================] - 0s 19ms/step - loss: 395968.8438 - val_loss: 255449.9219
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 390782.0938 - val_loss: 250900.8281
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 385700.4688 - val_loss: 245766.2500
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 380720.0938 - val_loss: 240921.3594
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 202407.7500 - val_loss: 65572.5703
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 201070.1719 - val_loss: 64685.1562
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 194429.9531 - val_loss: 76351.5625
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 199179.7031 - val_loss: 58772.5664
Epoch 223/10000
1/1 [==============================] - 0s 15ms/step - loss: 193959.5000 - val_loss: 80106.1328
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 197374.5781 - val_loss: 54809.7148
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 194954.3281 - val_loss: 59870.3008
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 208576.8906 - val_loss: 62233.1328
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 192745.3281 - val_loss: 54645.5703
Epoch 228/10000
1

Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 151328.8281 - val_loss: 14153.9697
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 153782.8281 - val_loss: 11773.3359
Epoch 295/10000
1/1 [==============================] - 0s 15ms/step - loss: 160484.0625 - val_loss: 26327.9941
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 155346.7500 - val_loss: 11575.9951
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 160354.9375 - val_loss: 19653.8008
Epoch 298/10000
1/1 [==============================] - 0s 15ms/step - loss: 152936.5000 - val_loss: 16921.7168
Epoch 299/10000
1/1 [==============================] - 0s 15ms/step - loss: 121037.1016 - val_loss: 23473.1055
Epoch 300/10000
1/1 [==============================] - 0s 15ms/step - loss: 150798.2500 - val_loss: 12082.5498
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 159207.5000 - val_loss: 17974.7168
E

...

Epoch 1123/10000
1/1 [==============================] - 0s 14ms/step - loss: 135486.4219 - val_loss: 12.1392
Epoch 1124/10000
1/1 [==============================] - 0s 15ms/step - loss: 135438.8906 - val_loss: 0.0064
Epoch 1125/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.7500 - val_loss: 0.8373
Epoch 1126/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1875 - val_loss: 0.4023
Epoch 1127/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.5053
Epoch 1128/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.4720
Epoch 1129/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.4784
Epoch 1130/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1406 - val_loss: 0.4733
Epoch 1131/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1406 - val_loss: 0.4720
Epoch 1132/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 137151.1406 - val_loss: 7.1181
Epoch 1199/10000
1/1 [==============================] - 0s 17ms/step - loss: 135435.5938 - val_loss: 0.0017
Epoch 1200/10000
1/1 [==============================] - 0s 18ms/step - loss: 135431.4531 - val_loss: 0.5141
Epoch 1201/10000
1/1 [==============================] - 0s 18ms/step - loss: 135431.1406 - val_loss: 0.2544
Epoch 1202/10000
1/1 [==============================] - 0s 18ms/step - loss: 135431.1094 - val_loss: 0.3176
Epoch 1203/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1094 - val_loss: 0.2989
Epoch 1204/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1094 - val_loss: 0.3036
Epoch 1205/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1094 - val_loss: 0.3020
Epoch 1206/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1094 - val_loss: 0.3020
Epoch 1207/10000
1/1 [=======================

Epoch 1274/10000
1/1 [==============================] - 0s 15ms/step - loss: 135446.1719 - val_loss: 0.1404
Epoch 1275/10000
1/1 [==============================] - 0s 14ms/step - loss: 135432.3281 - val_loss: 1.0256
Epoch 1276/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.2344 - val_loss: 0.3867
Epoch 1277/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1406 - val_loss: 0.5309
Epoch 1278/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1406 - val_loss: 0.4848
Epoch 1279/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.4940
Epoch 1280/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.4879
Epoch 1281/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1406 - val_loss: 0.4858
Epoch 1282/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1406 - val_loss: 0.4835
Epoch 1283/10000
1/1 [======

1/1 [==============================] - 0s 14ms/step - loss: 131137.5625 - val_loss: 37.3513
Epoch 1350/10000
1/1 [==============================] - 0s 14ms/step - loss: 135462.2031 - val_loss: 1.0509
Epoch 1351/10000
1/1 [==============================] - 0s 14ms/step - loss: 132500.3125 - val_loss: 90.6215
Epoch 1352/10000
1/1 [==============================] - 0s 15ms/step - loss: 135510.7969 - val_loss: 3.6347
Epoch 1353/10000
1/1 [==============================] - 0s 15ms/step - loss: 135437.3594 - val_loss: 1.6689
Epoch 1354/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.6094 - val_loss: 0.1563
Epoch 1355/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1562 - val_loss: 0.4167
Epoch 1356/10000
1/1 [==============================] - 0s 15ms/step - loss: 135431.1250 - val_loss: 0.3300
Epoch 1357/10000
1/1 [==============================] - 0s 14ms/step - loss: 135431.1094 - val_loss: 0.3522
Epoch 1358/10000
1/1 [=====================

Epoch 1425/10000
1/1 [==============================] - 0s 15ms/step - loss: 133577.7500 - val_loss: 101.8912
Epoch 1426/10000
1/1 [==============================] - 0s 14ms/step - loss: 131807.3281 - val_loss: 64.6167
Epoch 1427/10000
1/1 [==============================] - 0s 14ms/step - loss: 135484.9531 - val_loss: 1.8274
Epoch 1428/10000
1/1 [==============================] - 0s 25ms/step - loss: 135568.4219 - val_loss: 238.3466
Epoch 1428: early stopping


In [24]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[2022.411](test_target) - [[2018.833]](prediction) = 3.577992187500058


In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.969522,-1.359398,-1.050601,-0.981735,-1.252557,-1.264163,-1.233150,1.148205,-1.122043,-1.121418,...,1.412346,-0.449079,0.959570,1.349075,1.396668,1.302745,0.665154,-1.232563,-1.223094,1.316781
1,-0.406879,0.342020,-0.294564,-0.390692,0.057661,0.083064,0.016987,0.140893,-0.184474,-0.185492,...,-0.643250,1.385895,0.419892,-0.307116,-0.890660,-0.174780,0.748246,0.015789,-0.003295,-0.211662
2,1.376400,1.017379,1.345165,1.372427,1.194896,1.181098,1.216163,-1.289098,1.306518,1.306910,...,-0.769096,-0.936816,-1.379462,-1.041959,-0.506008,-1.127966,-1.413400,1.216774,1.226389,-1.105119


train_target:


,Pernambuco - Consumo de Cimento (t)
0,1878.938
1,2406.931
2,2496.113


test_input:


...

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,-2.260447,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739


test_target:


,Pernambuco - Consumo de Cimento (t)
12,2022.411


1/1 [==============================] - 0s 36ms/step
Error: 3.577992187500058




[2496.106689453125,
 2705.98779296875,
 2880.566162109375,
 2983.61279296875,
 2697.44677734375,
 2364.608154296875,
 1975.4654541015625,
 1647.99853515625,
 1963.17041015625,
 2018.8330078125]

In [27]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2496.106689453125,
 2705.98779296875,
 2880.566162109375,
 2983.61279296875,
 2697.44677734375,
 2364.608154296875,
 1975.4654541015625,
 1647.99853515625,
 1963.17041015625,
 2018.8330078125]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

233.42072953404022

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.10225994541190002